In [678]:
import numpy as np 
import torch 
import pandas as pd
import yfinance as yf
from arch import arch_model
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from scipy.optimize import minimize
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pytorch_lightning as pl


In [679]:
def generate_ground_garch(omega, alpha, beta, n=1000):
    
    am = arch_model(None, mean='Zero', vol='GARCH', p=1, q=1, power = 2) #Остатки просто получаются умножением волатильности на кси ~N(0,1)
    params = np.array([omega, alpha, beta])
    am_data = am.simulate(params, n)

    return am_data['data'].to_numpy()*100, am_data['volatility'].to_numpy()*100

In [680]:
omega, alpha, beta = 0.2, 0.2, 0.3

In [681]:
class CustomSyntDataset(Dataset):

    def __init__(self, omega, alpha, beta, n =1000):
        self.omega = omega
        self.alpha = alpha
        self.beta = beta
        self.n = n
        self.residuals, self.volatility = generate_ground_garch(omega, alpha, beta, n)

        self.inputs = np.column_stack([
            np.full_like(self.residuals, 1),
            np.square(self.residuals),
            np.square(self.volatility)
        ])
        
        self.outputs = np.square(np.roll(self.volatility, -1))

        self.inputs = torch.Tensor(self.inputs)

        self.outputs = torch.Tensor(self.outputs)

    def __len__(self):
        return self.n -1 
    
    def __getitem__(self, index):
        return self.inputs[index], self.outputs[index], self.inputs[index+1, 1]

In [682]:
ds = CustomSyntDataset(omega=omega, alpha=alpha, beta = beta)

In [683]:
for i in range(1, 5):
    print(ds[i])

(tensor([1.0000e+00, 2.0171e+03, 4.6836e+03]), tensor(3808.5059), tensor(952.6502))
(tensor([1.0000e+00, 9.5265e+02, 3.8085e+03]), tensor(3333.0818), tensor(14158.3496))
(tensor([1.0000e+00, 1.4158e+04, 3.3331e+03]), tensor(5831.5942), tensor(18.8512))
(tensor([1.0000e+00, 1.8851e+01, 5.8316e+03]), tensor(3753.2485), tensor(905.2452))


In [684]:
dl = DataLoader(ds, batch_size = 64, shuffle= False, drop_last=True)

In [685]:
class GarchNN(torch.nn.Module):
    def __init__(self):
        super(GarchNN, self).__init__()
        self.model =nn.Sequential(
            nn.Flatten(),
            nn.Linear(3,1, bias=False)
        )

    def forward (self, x):
        return self.model(x).squeeze(-1)

In [686]:
class NLoss(torch.nn.Module):
    def __init__(self):
        super(NLoss, self).__init__()

    def forward(self, pred_volatility, target_resid):
        left_side = torch.log(pred_volatility)/2
        right_side = (target_resid/pred_volatility) /2

        return (left_side + right_side).sum()
                

In [687]:
model = GarchNN()
optimizer = torch.optim.Adam(model.parameters(), lr =1e-1)
criterion = NLoss()
#criterion = nn.MSELoss()

In [688]:
num_epochs = 1000

for epochs in tqdm(range(num_epochs), desc="Training"):
    epoch_loss =0.0
    model.train()

    for inputs, targets, resids in dl:
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, resids)
        loss.backward()
        optimizer.step()

        epoch_loss+=loss.item()

    avg_loss = epoch_loss/len(dl)

    tqdm.write(f"Epoch {epochs+1}/{num_epochs} | Loss: {avg_loss:.4f}")
    

Training:   0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 1/1000 | Loss: 306.0074
Epoch 2/1000 | Loss: 299.5230
Epoch 3/1000 | Loss: 297.1282
Epoch 4/1000 | Loss: 296.8581
Epoch 5/1000 | Loss: 296.9336
Epoch 6/1000 | Loss: 296.8356
Epoch 7/1000 | Loss: 296.8925
Epoch 8/1000 | Loss: 296.8554
Epoch 9/1000 | Loss: 296.8816
Epoch 10/1000 | Loss: 296.8711
Epoch 11/1000 | Loss: 296.8813
Epoch 12/1000 | Loss: 296.8809
Epoch 13/1000 | Loss: 296.8849
Epoch 14/1000 | Loss: 296.8871
Epoch 15/1000 | Loss: 296.8894
Epoch 16/1000 | Loss: 296.8916
Epoch 17/1000 | Loss: 296.8936
Epoch 18/1000 | Loss: 296.8955
Epoch 19/1000 | Loss: 296.8973
Epoch 20/1000 | Loss: 296.8990
Epoch 21/1000 | Loss: 296.9006
Epoch 22/1000 | Loss: 296.9022
Epoch 23/1000 | Loss: 296.9037
Epoch 24/1000 | Loss: 296.9051
Epoch 25/1000 | Loss: 296.9065
Epoch 26/1000 | Loss: 296.9078
Epoch 27/1000 | Loss: 296.9091
Epoch 28/1000 | Loss: 296.9103
Epoch 29/1000 | Loss: 296.9115
Epoch 30/1000 | Loss: 296.9127
Epoch 31/1000 | Loss: 296.9138
Epoch 32/1000 | Loss: 296.9149
Epoch 33/1000 | L

In [689]:
print(torch.isnan(ds.inputs).any(), torch.isinf(ds.inputs).any())
print(torch.isnan(ds.outputs).any(), torch.isinf(ds.outputs).any())

tensor(False) tensor(False)
tensor(False) tensor(False)


In [690]:
def get_model_weights(model):
    weights = {}
    for name, param in model.named_parameters():
        weights[name] = param.data.clone().cpu().numpy()
    return weights

model_weights = get_model_weights(model)

In [691]:
model_weights

{'model.1.weight': array([[1.05148224e+02, 8.56708586e-02, 9.21257317e-01]], dtype=float32)}

In [692]:
test_variance = np.random.standard_normal()

In [693]:
test_cases = [
    # pred_vol, target_resid (все значения > 1)
    (torch.tensor([58]), torch.tensor([1])),
    (torch.tensor([180]), torch.tensor([145])),
    (torch.tensor([300]), torch.tensor([290])),

]

criterion = NLoss()

print("Тестирование NLoss для значений > 1:\n")
print("| pred_vol | target_resid | log(pred)/2 | resid/(2*pred) | total_loss |")
print("|----------|--------------|-------------|----------------|------------|")

for pred, target in test_cases:
    log_term = 0.5 * torch.log(pred)
    ratio_term = target / (2 * pred)
    loss = criterion(pred, target)
    
    print(f"| {pred.item():>8.2f} | {target.item():>12.2f} | {log_term.item():>11.4f} | {ratio_term.item():>14.4f} | {loss.item():>10.4f} |")

Тестирование NLoss для значений > 1:

| pred_vol | target_resid | log(pred)/2 | resid/(2*pred) | total_loss |
|----------|--------------|-------------|----------------|------------|
|    58.00 |         1.00 |      2.0302 |         0.0086 |     2.0388 |
|   180.00 |       145.00 |      2.5965 |         0.4028 |     2.9993 |
|   300.00 |       290.00 |      2.8519 |         0.4833 |     3.3352 |
